<a href="https://colab.research.google.com/github/JTalentTwoPointO/Automation/blob/master/dl_comp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from tensorflow import keras
import os
import glob
from PIL import Image

# Define data directories
train_dir = "/content/drive/MyDrive/comp2_open/train"
val_dir = "/content/drive/MyDrive/comp2_open/val"
test_dir = "/content/drive/MyDrive/comp2_open/test"

# Define file pattern and image size
file_pattern = "*.jpg"
image_size = (128, 128)

# Load data
def load_data(data_dir):
    data = []
    for file in os.listdir(data_dir):
        img = Image.open(data_dir + "/" + file)
        img = img.resize(image_size)
        img = np.array(img)
        data.append(img)
    return np.array(data)

train_data = load_data(train_dir)
val_data = load_data(val_dir)
test_data = load_data(test_dir)
train_labels = pd.read_csv("/content/drive/MyDrive/Comp2 /train_labels_ex2.csv")["label"].values
val_labels = pd.read_csv("/content/drive/MyDrive/Comp2 /val_labels_ex2.csv")["label"].values

# Print shape of data
print("Train data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Test data shape:", test_data.shape)







Train data shape: (614, 128, 128)
Validation data shape: (133, 128, 128)
Test data shape: (133, 128, 128)


In [ ]:
# Normalize data
train_data = train_data / 255.0
val_data = val_data / 255.0
test_data = test_data / 255.0

# Flatten data
train_data = train_data.reshape(train_data.shape[0], -1)
val_data = val_data.reshape(val_data.shape[0], -1)
test_data = test_data.reshape(test_data.shape[0], -1)

In [ ]:
# KMeans clustering
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(train_data)
train_clusters = kmeans.predict(train_data)
val_clusters = kmeans.predict(val_data)
test_clusters = kmeans.predict(test_data)

# Reshape train_clusters to have shape (num_samples, 1)
train_clusters = np.reshape(train_clusters, (train_clusters.shape[0], 1))

# Stack clusters to data
train_data = np.concatenate([train_data, train_clusters], axis=1)
val_data = np.concatenate([val_data, val_clusters.reshape((val_clusters.shape[0], 1))], axis=1)
test_data = np.concatenate([test_data, test_clusters.reshape((test_clusters.shape[0], 1))], axis=1)


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
# Define the autoencoder model
input_shape = train_data.shape[1:]
model = keras.models.Sequential([
    keras.layers.Dense(256, activation="relu", input_shape=input_shape),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    #ToDo - consider smaller bottle neck
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(np.prod(input_shape)),
    keras.layers.Reshape(input_shape)
])
model.compile(loss="mse", optimizer="adam")
model.fit(train_data, train_data, epochs=50, batch_size=32, validation_data=(val_data, val_data))







Epoch 1/50
20/20 [==============================] - 9s 19ms/step - loss: 0.2040 - val_loss: 0.0231
Epoch 2/50
20/20 [==============================] - 0s 8ms/step - loss: 0.0185 - val_loss: 0.0141
Epoch 3/50
20/20 [==============================] - 0s 8ms/step - loss: 0.0145 - val_loss: 0.0132
Epoch 4/50
20/20 [==============================] - 0s 8ms/step - loss: 0.0141 - val_loss: 0.0131
Epoch 5/50
20/20 [==============================] - 0s 8ms/step - loss: 0.0141 - val_loss: 0.0131
Epoch 6/50
20/20 [==============================] - 0s 8ms/step - loss: 0.0141 - val_loss: 0.0131
Epoch 7/50
20/20 [==============================] - 0s 8ms/step - loss: 0.0141 - val_loss: 0.0132
Epoch 8/50
20/20 [==============================] - 0s 8ms/step - loss: 0.0141 - val_loss: 0.0131
Epoch 9/50
20/20 [==============================] - 0s 8ms/step - loss: 0.0141 - val_loss: 0.0132
Epoch 10/50
20/20 [==============================] - 0s 8ms/step - loss: 0.0142 - val_loss: 0.0132
Epoch 11/50
20/20 

In [ ]:
encoder = keras.models.Model(inputs=model.input, outputs=model.layers[-2].output)
train_data = encoder.predict(train_data)
val_data = encoder.predict(val_data)
test_data = encoder.predict(test_data)

5/5 [==============================] - 0s 3ms/step


In [ ]:
# Add Gaussian noise
noise_factor = 0.1
train_data_noisy = train_data + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=train_data.shape)
val_data_noisy = val_data + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=val_data.shape)
test_data_noisy = test_data + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=test_data.shape)

# Clip noisy data to [0, 1]
train_data_noisy = np.clip(train_data_noisy, 0., 1.)
val_data_noisy = np.clip(val_data_noisy, 0., 1.)
test_data_noisy = np.clip(test_data_noisy, 0., 1.)

In [ ]:
# Define the neural network model
model = keras.models.Sequential([
    keras.layers.Dense(256, activation="relu", input_shape=train_data_noisy.shape[1:]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_data_noisy, train_labels, epochs=100, batch_size=32, validation_data=(val_data_noisy, val_labels))


model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_data_noisy, train_labels, epochs=100, batch_size=32, validation_data=(val_data_noisy, val_labels))



Epoch 1/100
20/20 [==============================] - 3s 18ms/step - loss: 1.0966 - accuracy: 0.5407 - val_loss: 0.8395 - val_accuracy: 0.5188
Epoch 2/100
20/20 [==============================] - 0s 12ms/step - loss: 0.9665 - accuracy: 0.5277 - val_loss: 1.6537 - val_accuracy: 0.5188
Epoch 3/100
20/20 [==============================] - 0s 9ms/step - loss: 0.9645 - accuracy: 0.5049 - val_loss: 0.8096 - val_accuracy: 0.4812
Epoch 4/100
20/20 [==============================] - 0s 10ms/step - loss: 0.7135 - accuracy: 0.4788 - val_loss: 0.7536 - val_accuracy: 0.5188
Epoch 5/100
20/20 [==============================] - 0s 9ms/step - loss: 0.7566 - accuracy: 0.4723 - val_loss: 0.7157 - val_accuracy: 0.5188
Epoch 6/100
20/20 [==============================] - 0s 12ms/step - loss: 0.7620 - accuracy: 0.4886 - val_loss: 0.7165 - val_accuracy: 0.5188
Epoch 7/100
20/20 [==============================] - 0s 9ms/step - loss: 0.7103 - accuracy: 0.5130 - val_loss: 0.7009 - val_accuracy: 0.4812
Epoch 8/1

In [ ]:
# Evaluate model
test_labels = model.predict(test_data_noisy)
test_labels = (test_labels > 0.5).astype(int)

# Load test file names and remove file extension
test_files = []
for file in os.listdir(test_dir):
    filename, extension = os.path.splitext(file)
    test_files.append(filename)

# Create dataframe with file names and predicted labels
test_labels_df = pd.DataFrame({"id": test_files, "label": test_labels.squeeze()})

# Save labels to csv
test_labels_df.to_csv("/content/test_labels_ex2.csv", index=False)



5/5 [==============================] - 0s 3ms/step
